In [14]:
!pip install 'shimmy>=0.2.1'

In [15]:
import gym
from stable_baselines3 import PPO
import time

class ThermalManagementEnv(gym.Env):
    def __init__(self, target_temp):
        super(ThermalManagementEnv, self).__init__()
        self.target_temp = target_temp
        self.current_temp = 25.0  # Initial temperature
        self.max_fan_speed = 10   # Maximum fan speed
        self.min_fan_speed = 0    # Minimum fan speed
        self.action_space = gym.spaces.Discrete(self.max_fan_speed + 1)
        self.observation_space = gym.spaces.Box(low=0, high=100, shape=(1,), dtype=float)

    def step(self, action):
        fan_speed = action
        self.current_temp += (fan_speed / 10) - 0.5  # Simplified temperature dynamics
        reward = -abs(self.current_temp - self.target_temp)
        done = abs(self.current_temp - self.target_temp) < 0.5
        return self.current_temp, reward, done, {}

    def reset(self):
        self.current_temp = 25.0
        return self.current_temp

target_temp = 22.0
env = ThermalManagementEnv(target_temp)

model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)
model.save("thermal_management_model")

loaded_model = PPO.load("thermal_management_model")

obs = env.reset()
while True:
    action, _ = loaded_model.predict([obs])  # No need to reshape the observation
    obs, _, done, _ = env.step(action)
    print("Current Temperature:", obs)
    if done:
        print("Target temperature reached!")
        break
    time.sleep(1)




Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 106      |
|    ep_rew_mean     | -332     |
| time/              |          |
|    fps             | 668      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 177         |
|    ep_rew_mean          | -995        |
| time/                   |             |
|    fps                  | 500         |
|    iterations           | 2           |
|    time_elapsed         | 8           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.016696088 |
|    clip_fraction        | 0.0789      |
|    clip_range           | 0.2         |
|    entropy_loss   